In [1]:
from pydub import *
import numpy as np
import pandas as pd
from py_apps.utils.classify_app import ClassifyApp
import networkx as nx
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import json
from sklearn.preprocessing import normalize
from models import ClassifyAudioApp
import argparse
from py_apps.utils.common_utils import read_network
from py_apps.utils.common_utils import load_json_arg
from py_apps.utils.neuro_help import *
from models import ClassifyAudioApp

In [2]:
background = "data/debugData/background/background_0a075411-0c97-4ffe-8d0d-c25a79580396.wav"
gunshot = "data/debugData/gunshot/gunshot_0d50a744-55a3-42f0-826c-9c0d67ebd512.wav"
index = 'data/debugData/index.csv'

In [3]:
i = pd.read_csv(index, header=None).to_numpy()

In [4]:
len(i)

100

In [5]:
legnths = []
for file in i[:, 0]:
    t = len(AudioSegment.from_file(file, 'wav',duration=60).get_array_of_samples())
    if t not in legnths:
        legnths.append(t)

In [6]:
x = []
for file in i[:, 0]:
    sample = AudioSegment.from_file(file, 'wav',duration=60).get_array_of_samples()
    x.append(sample)

In [11]:
x = np.array(x)
x = normalize(x)

In [12]:
x.shape

(100, 960000)

In [289]:
legnths

[960000]

In [290]:
a = AudioSegment.from_file(background, 'wav', duration=60)
b = AudioSegment.from_file(gunshot, 'wav', duration=60)

In [291]:
AudioSegment.from_file(background, 'wav', duration=60)

In [292]:
len(a)

5000

In [293]:
len(b)

5000

In [294]:
print(len(a.get_array_of_samples()))
print(len(b.get_array_of_samples()))

960000
960000


In [295]:
data = np.array([list(a.get_array_of_samples()), list(b.get_array_of_samples())])

In [296]:
np.mean(data)
np.std(data)


151732230.91831303

In [297]:
spiks = np.array(data)
spiks

array([[ -67698688, -107872256,  -24838144, ...,          0,          0,
                 0],
       [ -46268416,  -42205184,  -43384832, ...,  -15859712,          0,
                 0]])

In [298]:
# mean=[2.3009], std=[42.1936]
normSpikes = normalize(spiks)

In [299]:
spikTrains = np.expand_dims(normSpikes, axis=1)
spikTrains.shape

(2, 1, 960000)

In [300]:
labels = np.array([0,1])
labels.shape

(2,)

In [301]:
np.save('spikTrains', spikTrains)
np.save('spikTrains_labels', labels)


In [302]:
params = """
	-a train \
	--proc_params ./config/risp.json\
	--eons_params ./config/eons.json\
	--timeseries true \
	--app_type load \
	--data_np spikTrains.npy \
	--labels_np spikTrains_labels.npy \
	--encoder ./config/encoder.json \
	--processes 1 \
	--sim_time 960000 \
	--epochs 3 \
	--network_filename test_simple.json
    """

In [303]:
parser = argparse.ArgumentParser(description="Classification Application Driver")
parser.add_argument("--activity", "-a", required=True, type=str, choices=["train", "test"], help="activity to perform")
parser.add_argument("--network_filename", default=None, type=str, help="location to store the best network file produced if training or network to load if testing")
parser.add_argument("--sim_time", default=50, type=float, help="the simulation timefor each data instance")
parser.add_argument("--eons_params", default="config/eons.json", type=str, help="JSON file with EONS parameters")
parser.add_argument("--extra_eons_params", default="{}", type=str ,help="JSON file or JSON string updating EONS parameters from configuration file")
parser.add_argument("--epochs", default=50, type=int, help="epochs for eons")
parser.add_argument("--max_fitness", default=1e+06, type=float, help="max fitness for eons")
parser.add_argument("--processes", default=1, type=int, help="processes for EONS")
parser.add_argument("--test_seed", default=1234, type=int, help="testing seed")

print("Reading classification params")
add_class_arguments(parser)

print("Reading proc params")
add_proc_arguments(parser)

print("Reading encoder information")
add_coder_arguments(parser)

print("Reading printing params")
add_printing_arguments(parser)

args = parser.parse_args(params.split())

Reading classification params
Reading proc params
Reading encoder information
Reading printing params


In [304]:
 # Read in proc params as necessary
proc_params = load_json_arg(args.proc_params)
extra_proc_params = load_json_arg(args.extra_proc_params)
for k in extra_proc_params.keys():
    proc_params[k] = extra_proc_params[k]

# Read in EONS params
eons_params = load_json_arg(args.eons_params)

#Read in extra EONS params
extra_eons_params = load_json_arg(args.extra_eons_params)

for k in extra_eons_params.keys():
    eons_params[k] = extra_eons_params[k]
#print(eons_params)

proc_instantiation = get_proc_instantiation(args.proc_name)

# Instantiate data
X, y = setup_data(args)

config = setup_class_config(args)

In [305]:
X

array([[[-0.00033599, -0.00053537, -0.00012327, ...,  0.        ,
          0.        ,  0.        ]],

       [[-0.00077043, -0.00070277, -0.00072242, ..., -0.00026409,
          0.        ,  0.        ]]])

In [306]:
y

array([0, 1])

In [308]:
app = ClassifyAudioApp(config, X, y)

train_params = {}
train_params["eons_params"] = eons_params
train_params["num_epochs"] = args.epochs
train_params["num_processes"] = args.processes
train_params["fitness"] = args.fitness
train_start = time.time()
app.train(train_params, proc_instantiation, proc_params)
train_end = time.time()
elapsed = train_end-train_start
net = app.overall_best_net
net.prune()
print("Network size:", net.num_nodes(), net.num_edges())
print("Training Time:", elapsed)

Epoch:   0     Time:   15.8     Best: 1.0    Num_Synapses: 40
Epoch:   1     Time:   12.6     Best: 1.0    Num_Synapses: 40


KeyboardInterrupt: 